In [34]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime, timedelta

# Definición de base de conocimiento para las reglas lógicas
base_conocimiento = "planeacion_gdb2.csv"

def read_stations_data(base_conociemiento):
    stations_df = pd.read_csv(base_conociemiento)
    return {row['NOMBRE']: (float(row['X']), float(row['Y'])) for _, row in stations_df.iterrows()}



    tiempos_ida_vuelta = {
    ("Tricentenario", "Caribe"): 10,
    ("Caribe", "Hospital"): 15,
    ("Hospital", "Parque Berrío"): 20,
    ("Parque Berrío", "San Antonio"): 25,
    ("Hospital", "Bello"): 35,
    ("Bello", "Hospital"): 15,  # Agregar tiempo de vuelta para la combinación Bello-Hospital
    ("Niquía", "Tricentenario"): 5,  # Agregar tiempo de ida para la combinación Niquía-Tricentenario
    ("Niquía", "Bello"): 30  # Agregar tiempo de ida para la combinación Niquía-Bello
}  # Diccionario para almacenar los tiempos de ida y vuelta

def calcular_distancia(coord1, coord2):
    R = 6371  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(radians, (*coord1, *coord2))

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    distancia = R * c

    return distancia

def calcular_hora_llegada(distancia, velocidad_promedio, tiempo_ida_vuelta):
    hora_actual = datetime.now()
    tiempo_estimado_minutos = (distancia / velocidad_promedio) * 60  # Convertir a minutos
    tiempo_estimado = timedelta(minutes=tiempo_estimado_minutos + tiempo_ida_vuelta)  # Utilizar timedelta en minutos
    hora_llegada = hora_actual + tiempo_estimado
    return tiempo_estimado_minutos, hora_llegada

def calcular_velocidad_promedio(distancia, tiempo_total):
    tiempo_ida_vuelta = tiempo_total / 2  # Dividir el tiempo total entre ida y vuelta
    velocidad_promedio = distancia / (tiempo_ida_vuelta / 60)  # Convertir minutos a horas
    return velocidad_promedio

def obtener_combinaciones_tiempos():
    combinaciones_tiempos = {}
    for origen, destino in tiempos_ida_vuelta.keys():
        tiempo_ida_vuelta = tiempos_ida_vuelta[(origen, destino)]
        combinaciones_tiempos[(origen, destino)] = tiempo_ida_vuelta
        combinaciones_tiempos[(destino, origen)] = tiempo_ida_vuelta
    return combinaciones_tiempos


def ir_a_estacion(estaciones_data, estacion_actual, estacion_destino, velocidad_promedio_km_h):
    if estacion_actual not in estaciones_data or estacion_destino not in estaciones_data:
        return "Estación de origen o destino no válida."
    distancia_km = calcular_distancia(estaciones_data[estacion_actual], estaciones_data[estacion_destino])

    #if isinstance(distancia_km, str):
        #return distancia_km

    combinaciones_tiempos = obtener_combinaciones_tiempos()

    tiempo_ida_vuelta = 0
    tiempo_estimado_minutos = 0  # Asignar un valor predeterminado
    if (estacion_actual, estacion_destino) in combinaciones_tiempos:
        tiempo_ida_vuelta = combinaciones_tiempos[(estacion_actual, estacion_destino)]
        velocidad_promedio_km_h = 80
        tiempo_estimado_minutos, hora_llegada = calcular_hora_llegada(distancia_km, velocidad_promedio_km_h, tiempo_ida_vuelta)


    return f'La distancia entre {estacion_actual} y {estacion_destino} es de {distancia_km} km.\n' \
           f'La velocidad promedio es de: {velocidad_promedio_km_h}.\n' \
           f'Minutos estimados en llegar: {round(tiempo_estimado_minutos, 2)}.\n'




# Leer datos de las estaciones desde el archivo CSV
estaciones_data = read_stations_data('planeacion_gdb2.csv')

# Ejemplo de uso
estacion_origen = input("Ingresa la estación de origen: ")
estacion_destino = input("Ingresa la estación de destino: ")
velocidad_promedio_km_h = 80

resultado = ir_a_estacion(estaciones_data, estacion_origen, estacion_destino, velocidad_promedio_km_h)
print(resultado)


Ingresa la estación de origen: Bello
Ingresa la estación de destino: Hospital
La distancia entre Bello y Hospital es de 7.487134145771767 km.
La velocidad promedio es de: 80.
Minutos estimados en llegar: 5.62.

